# AGO Edit Metadata

This notebook will edit metadata (specifically the source) for ArcGIS Online resources

In [1]:
# import modules
import arcgis, getpass

# get password
password = getpass.getpass()
username = ""

# connect to ArcGIS Online
gis = arcgis.gis.GIS("https://www.arcgis.com", username, password)

········


Set variables like the item to find & various pieces to update in the metadata if desired. 

In [21]:
title = "Wakarusa Watershed PDF Downloads"
owner = username

# set new metadata info
lineage = "From NRCS"
dateInfo = "20160301"
credits = "NRCS"

[<Item title:"Wakarusa Watershed PDF Downloads" type:Web Map owner:KansasGIS>]

Get an item, various query options available

In [ ]:
#items = gis.content.search(query='title:%s owner:%s'% (title, owner), item_type='Feature Layer')
items = gis.content.search(query='title:%s owner:%s'% (title, owner))
#items = gis.content.search(query='owner:KansasGIS source:kansas state government gis')
items

Define terms of use

In [ ]:
blurb = ""
#blurb = """The State of Kansas Geographic Information Systems Policy Board's Data Access and Support Center (DASC) and its representatives make no warranty or representation, either expressed or implied, with respect to the digital data and their documentation, their quality, performance, merchantability, or fitness for a particular purpose. The digital data are distributed on as-is basis, and the user assumes all risk to their quality, the results obtained from their use, and the performance of the data. In no event will the State of Kansas Geographic Information Systems Policy Board or its representatives be liable for any direct, indirect, special, incidental or consequential damages resulting from and defect in the State of Kansas DASC hosted digital data or in their documentation. This disclaimer of warranty is exclusive and in lieu of all others, oral or written, express or implied. No agent or employee is authorized to make any modification, extension, or addition to this warranty."""

Loop through items from the returned list and append Metadata to typeKeywords. Without this step, Metadata isn't enabled.

In [5]:
for targetItem in items:
    

    existingKeywords = targetItem.typeKeywords

    if "Metadata" not in existingKeywords:

        newKeywords = existingKeywords
        newKeywords.append("Metadata")

        print(newKeywords)

        prop = {"typeKeywords":newKeywords}
        targetItem.update(item_properties=prop)

['ArcGIS Online', 'Explorer Web Map', 'Map', 'Online Map', 'Web Map', 'Metadata']


Run fresh search to get the item again, double check typeKeywords

In [12]:
index = 0 # change index as necessary to get the proper item, can run this and subsequent cells often
items = gis.content.search(query='title:%s owner:%s'% (title, owner))
targetItem = items[index]
targetItem.typeKeywords
oldTags = targetItem.tags
oldTags

['transportation', 'Alternative', 'Fuels', 'Kansas']

Create a copy of the metadata template for this particular layer

In [13]:
from shutil import copy2
from os.path import join, exists
from os import remove

# copy metadata file

# template metadata
templateMetadata = r"C:\myPath\template_metadata_AGO2.xml"

# output metadata
# folder where you want to save a copy of the metadata to be uploaded to the ArcGIS Online item
folder = r"" 
metadataName = targetItem.title.replace(" ", "_").replace("Data:_","") + "_metadata_AGO.xml"
metadataFile = join(folder, metadataName)

if exists(metadataFile):
    remove(metadataFile)

copy2(templateMetadata, metadataFile)

'C:\\Users\\kristen\\Documents\\Data\\AGO\\metadata\\toUpload\\Alternative_Fuel_Stations_metadata_AGO.xml'

Edit metadata file, edit the updateDict based on what fields you need to update

In [15]:
import xml.etree.ElementTree as etree
from xml.etree.ElementTree import parse
from xml.dom import minidom

# code for editing metadata file here

xmlKJ = parse(metadataFile)

#updateDict = {"CreaDate": dateInfo, "itemName": targetItem.title.replace("Data: ", ""),
#             "SyncDate": dateInfo, "ModDate": dateInfo, "resTitle": targetItem.title.replace("Data: ", ""),
#             "exDesc": targetItem.snippet, "idPurp": targetItem.snippet, "idAbs": targetItem.description,
#             "idCredit": credits,"mdDateSt": dateInfo, "statement": lineage}
updateDict = {"itemName": targetItem.title.replace("Data: ", ""),
             "resTitle": targetItem.title.replace("Data: ", ""),
             "exDesc": targetItem.snippet, "idPurp": targetItem.snippet, "idAbs": targetItem.description,
             "idCredit": credits, "rpOrgName": "Data Access and Support Center"}

for update in updateDict:
    newInfo = updateDict[update]
    fixItems = xmlKJ.getiterator(update)
    for fixItem in fixItems:
        fixItem.text = newInfo
        
xmlKJ.write(metadataFile)

Upload metadata file back into the targetItem

In [16]:
blankProp = {}

targetItem.update(item_properties=blankProp, metadata=metadataFile)

redoTags = {"tags": oldTags}

targetItem.update(item_properties=redoTags)

True